In [1]:
import pdb
from ssdmultibox.datasets import *

np.set_printoptions(precision=15)

In [2]:
dataset = TrainPascalDataset()

In [3]:
image_id, chw_im, gt_bbs, gt_cats = dataset[1]
image_id, chw_im.shape

(17, (3, 300, 300))

In [4]:
im = np.reshape(chw_im, (SIZE, SIZE, 3))
im.shape

(300, 300, 3)

In [5]:
ann = dataset.get_annotations()[image_id]
ann

{'image_path': '/Users/alelevier/data/JPEGImages/000017.jpg',
 'bbs': [[184, 61, 95, 138], [89, 77, 314, 259]],
 'cats': [14, 12]}

In [6]:
# understand how gt_overlaps and gt_idx are being set
# so it can be used with the "stacked" functions

In [19]:
bbs = ann['bbs']
im = open_image(ann['image_path'])
grid_size = 2
aspect_ratio = (1,1)

Bboxer.get_gt_overlap_and_idx(bbs, im, grid_size, aspect_ratio)

(tensor([1.9900, 0.1580, 0.2281, 1.9900], dtype=torch.float64),
 tensor([0, 1, 1, 1]))

In [20]:
overlaps = torch.tensor(Bboxer.get_iou(bbs, im, grid_size, aspect_ratio))
overlaps

tensor([[0.1364, 0.0871, 0.0159, 0.0106],
        [0.1463, 0.1580, 0.2281, 0.2477]], dtype=torch.float64)

In [21]:
overlaps.shape

torch.Size([2, 4])

In [24]:
prior_overlap, prior_idx = overlaps.max(1)
prior_overlap, prior_idx

(tensor([0.1364, 0.2477], dtype=torch.float64), tensor([0, 3]))

In [25]:
gt_overlap, gt_idx = overlaps.max(0)
gt_overlap, gt_idx

(tensor([0.1463, 0.1580, 0.2281, 0.2477], dtype=torch.float64),
 tensor([1, 1, 1, 1]))

In [33]:
gt_overlap[prior_idx] = 1.99
gt_overlap

tensor([1.9900, 0.1580, 0.2281, 1.9900], dtype=torch.float64)

In [34]:
gt_idx

tensor([1, 1, 1, 1])

In [42]:
for i,o in enumerate(prior_idx):
    print(i, o.item())
    gt_idx[o] = i
    
gt_idx

0 0
1 3


tensor([0, 1, 1, 1])

In [43]:
# get_stacked_iou

In [53]:
bbs_area = Bboxer.get_bbs_area(bbs, im)
bbs_area

array([0.073121993284493, 0.460923504273504])

In [51]:
grid_size, Bboxer.get_ancb_area(grid_size)

(2, 0.25)

In [92]:
anchor_bbs = Bboxer.stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
anchor_bbs

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ],
       [0. , 0. , 1. , 1. ]])

In [84]:
bbs

[[184, 61, 95, 138], [89, 77, 314, 259]]

In [86]:
np.square(4)

16

In [87]:
np.array(bbs).shape

(2, 4)

In [ ]:
def get_stacked_anchor_boxes_area(bbs_area, feature_maps, aspect_ratios):
    aspect_ratios = aspect_ratios or cls.aspect_ratios
    boxes = None
    for i in range(len(feature_maps)):
        grid_size = feature_maps[i]
        for aspect_ratio in aspect_ratios(grid_size):
            arr = np.repeat(
                Bboxer.get_ancb_area(grid_size)*aspect_ratios[0]*aspect_ratios[1],
                np.square(grid_size)*bbs.shape[0]
            ).reshape(-1, bbs.shape[0]) + bbs_area
            if not isinstance(boxes, np.ndarray):
                boxes = arr
            else:
                boxes = np.concatenate((boxes, arr))
                
    stacked_anchor_boxes = Bboxer.stacked_anchor_boxes(feature_maps, aspect_ratios)
    stacked_intersect = Bboxer.get_stacked_intersection(bbs, im, stacked_anchor_boxes)
    intersect = stacked_intersect.T.reshape(boxes.shape[0], -1)
    
    pdb.set_trace()

    return boxes - intersect
    

bbs_area = Bboxer.get_bbs_area(bbs, im)
get_stacked_anchor_boxes_area(bbs_area, feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1), (1,2)])

In [ ]:
%debug

In [177]:
get_stacked_anchor_boxes_area(bbs_area, feature_maps=[2], aspect_ratios=lambda grid_size:[(1,1), (1,2)])

array([[0.284339942002442, 0.620178311965812],
       [0.297221123321123, 0.613928311965812],
       [0.318062164224664, 0.578879166666667],
       [0.31974275030525 , 0.569784722222222],
       [0.258439072039072, 0.548062927350427],
       [0.258439072039072, 0.541812927350427],
       [0.314682921245421, 0.473943269230769],
       [0.314682921245421, 0.464848824786325]])

In [175]:
ann['cats']

[14, 12]

In [176]:
dataset.categories()[14]

'person'

In [129]:
grid_size = 2

bbs = np.array(bbs)

ret = np.repeat(
    Bboxer.get_ancb_area(grid_size), np.square(grid_size)*bbs.shape[0]
).reshape(-1, bbs.shape[0]) + bbs_area
ret, ret.shape

(array([[0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504]]), (4, 2))

In [133]:
grid_size = 1

ret2 = np.repeat(
    Bboxer.get_ancb_area(grid_size), np.square(grid_size)*bbs.shape[0]
).reshape(-1, bbs.shape[0]) + bbs_area
ret2, ret2.shape

(array([[1.073121993284493, 1.460923504273504]]), (1, 2))

In [149]:
np.square(grid_size)*bbs.shape[0]

2

In [151]:
bbs_area

array([0.073121993284493, 0.460923504273504])

In [136]:
np.concatenate((ret, ret2))

array([[0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [1.073121993284493, 1.460923504273504]])

In [144]:
stacked_anchor_boxes = Bboxer.stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
stacked_anchor_boxes, stacked_anchor_boxes.shape

(array([[0. , 0. , 0.5, 0.5],
        [0. , 0.5, 0.5, 1. ],
        [0.5, 0. , 1. , 0.5],
        [0.5, 0.5, 1. , 1. ],
        [0. , 0. , 1. , 1. ]]), (5, 4))

In [145]:
stacked_intersect = Bboxer.get_stacked_intersection(bbs, im, stacked_anchor_boxes)
stacked_intersect, stacked_intersect.shape

(array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
         0.003379242979243, 0.073121993284493],
        [0.090745192307692, 0.096995192307692, 0.132044337606838,
         0.141138782051282, 0.460923504273504]]), (2, 5))

In [146]:
stacked_intersect.reshape(-1, 2)

array([[0.038782051282051, 0.02590086996337 ],
       [0.005059829059829, 0.003379242979243],
       [0.073121993284493, 0.090745192307692],
       [0.096995192307692, 0.132044337606838],
       [0.141138782051282, 0.460923504273504]])

In [167]:
raw_ret = np.concatenate((ret, ret2)) - stacked_intersect.T.reshape(5, -1)
raw_ret

array([[0.284339942002442, 0.620178311965812],
       [0.297221123321123, 0.613928311965812],
       [0.318062164224664, 0.578879166666667],
       [0.31974275030525 , 0.569784722222222],
       [1.               , 1.               ]])

In [168]:
np.array([
        [0.284339942002442, 0.620178311965812],
        [0.297221123321123, 0.613928311965812],
        [0.318062164224664, 0.578879166666667],
        [0.31974275030525 , 0.569784722222222],
        [1.               , 1.               ]]) == raw_ret

array([[False,  True],
       [False,  True],
       [False, False],
       [False, False],
       [ True,  True]])

In [162]:
bbs_area

array([0.073121993284493, 0.460923504273504])

In [163]:
ret

array([[0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504]])

In [161]:
ret2

array([[1.073121993284493, 1.460923504273504]])

In [165]:
np.concatenate((ret, ret2))

array([[0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [1.073121993284493, 1.460923504273504]])

In [50]:
stacked_intersect = Bboxer.get_stacked_intersection(bbs, im, anchor_bbs)
stacked_intersect, stacked_intersect.shape

(array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
         0.003379242979243, 0.073121993284493],
        [0.090745192307692, 0.096995192307692, 0.132044337606838,
         0.141138782051282, 0.460923504273504]]), (2, 5))

In [73]:
bbs_area_union = (np.repeat(.25, 8).reshape(4,2) + bbs_area)
bbs_area_union, bbs_area_union.shape

(array([[0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504]]), (4, 2))

In [74]:
bbs_area

array([0.073121993284493, 0.460923504273504])

In [79]:
np.tile(bbs_area, 4).reshape(4,2) + .25

array([[0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504]])

In [80]:
np.tile(bbs_area, 4).reshape(4,2) + np.repeat(.25, 8).reshape(4,2)

array([[0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504]])

In [21]:
im = open_image(ann['image_path'])
scaled_bbs = Bboxer.scaled_fastai_bbs(ann['bbs'], im=im)
scaled_bbs

array([[0.167582417582418, 0.383333333333333, 0.543369963369963,
        0.577916666666667],
       [0.211538461538462, 0.185416666666667, 0.91974358974359 ,
        0.83625          ]])

In [11]:
# plot and make sure correct

In [12]:
from ssdmultibox.plotting import show_img, draw_rect

In [25]:
# ok

# resized_im = cv2.resize(im, (SIZE, SIZE))
# ax = show_img(resized_im)
# for bb in scaled_bbs:
#     draw_rect(ax, Bboxer.fastai_bb_to_pascal_bb(bb)*SIZE)

In [30]:
anc1 = Bboxer.anchor_corners(2, (1,1))
anc1

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ]])

In [32]:
anc2 = Bboxer.anchor_corners(1, (1,1))
anc2

array([[0., 0., 1., 1.]])

In [33]:
np.concatenate((anc1, anc2))

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ],
       [0. , 0. , 1. , 1. ]])

In [49]:
def stacked_anchor_boxes(feature_maps=FEATURE_MAPS, aspect_ratios=None):
    aspect_ratios = aspect_ratios or cls.aspect_ratios
    boxes = None
    for i in range(len(feature_maps)):
        grid_size = feature_maps[i]
        for aspect_ratio in aspect_ratios(grid_size):
            arr = np.clip(Bboxer.anchor_corners(grid_size, aspect_ratio), 0, 1)
            if not isinstance(boxes, np.ndarray):
                boxes = arr
            else:
                boxes = np.concatenate((boxes, arr))
    return boxes

anchor_bbs = stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
anchor_bbs

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ],
       [0. , 0. , 1. , 1. ]])

In [50]:
anchor_bbs.shape

(5, 4)

In [113]:
def get_stacked_intersection(bbs, im, anchor_bbs):
    bbs = Bboxer.scaled_fastai_bbs(bbs, im)
    bbs_count = anchor_bbs.shape[0]
    bbs16 = np.reshape(np.tile(bbs, bbs_count), (-1,bbs_count,4))
    intersect = np.minimum(
        np.maximum(anchor_bbs[:,:2], bbs16[:,:,:2]) - \
        np.minimum(anchor_bbs[:,2:], bbs16[:,:,2:]), 0)
    return intersect[:,:,0] * intersect[:,:,1]

anchor_bbs = stacked_anchor_boxes(
    feature_maps=[1], aspect_ratios=lambda grid_size:[(1,1)])

ann = dataset.get_annotations()[12]

get_stacked_intersection(ann['bbs'], im, anchor_bbs)

array([[0.201791531531531]])

In [117]:
ann = dataset.get_annotations()[image_id]

anchor_bbs = stacked_anchor_boxes(feature_maps=[1], aspect_ratios=lambda grid_size:[(1,1)])

stacked_intersect = get_stacked_intersection(ann['bbs'], open_image(ann['image_path']), anchor_bbs)
stacked_intersect.shape, stacked_intersect

((2, 1), array([[0.073121993284493],
        [0.460923504273504]]))

In [116]:
anchor_bbs

array([[0., 0., 1., 1.]])

In [104]:
anchor_bbs, anchor_bbs.shape

(array([[0., 0., 1., 1.]]), (1, 4))

In [98]:
anchor_bbs = stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
anchor_bbs

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ],
       [0. , 0. , 1. , 1. ]])

In [100]:
stacked_intersect = get_stacked_intersection(ann['bbs'], im, anchor_bbs)
stacked_intersect.shape, stacked_intersect

((2, 5), array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
         0.003379242979243, 0.073121993284493],
        [0.090745192307692, 0.096995192307692, 0.132044337606838,
         0.141138782051282, 0.460923504273504]]))

In [110]:
ann = dataset.get_annotations()[12]
bbs = ann['bbs']
im = open_image(ann['image_path'])
anchor_bbs = stacked_anchor_boxes(
    feature_maps=[1], aspect_ratios=lambda grid_size:[(1.,1.)])

ret = get_stacked_intersection(bbs, im, anchor_bbs)
ret

array([[0.201791531531531]])

In [111]:
anchor_bbs = stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
ret = get_stacked_intersection(bbs, im, anchor_bbs)
ret

array([[0.040225225225225, 0.04206006006006 , 0.058420720720721,
        0.061085525525526, 0.201791531531531]])